<a href="https://colab.research.google.com/github/polydeuces32/s-p500-predicter-on-Python-/blob/main/NeuroSPYder(LSTM)_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance


# 📊 What You’ll Get:
Sharpe Ratio 📈

Max Drawdown 📉

Final Balance 💰

Trade Log Table **🧾**

In [ ]:
# Re-run everything after code execution reset

# Step 1: Install Required Libraries
!pip install yfinance tensorflow scikit-learn matplotlib --quiet


In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional, BatchNormalization

# Step 1: Download SPY Data
data = yf.download('SPY', start='2018-01-01', interval='1d')[['Close']]
data['Close'] = data['Close'].ffill()

# Step 2: Add Rolling Volatility
data['Volatility'] = data['Close'].rolling(window=10).std()
data.dropna(inplace=True)

# Step 3: Normalize Prices
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

# Step 4: Prepare Sequences
sequence_length = 60
X, y = [], []
for i in range(sequence_length, len(scaled_data) - 1):
    X.append(scaled_data[i - sequence_length:i, 0])
    y.append(scaled_data[i + 1, 0])
X, y = np.array(X), np.array(y)
X = X.reshape((X.shape[0], X.shape[1], 1))

# Step 5: Build the LSTM Model
model = Sequential([
    Bidirectional(LSTM(units=64, return_sequences=True, input_shape=(X.shape[1], 1))),
    Dropout(0.3),
    BatchNormalization(),
    LSTM(units=64, return_sequences=True),
    Dropout(0.3),
    BatchNormalization(),
    LSTM(units=32),
    Dropout(0.2),
    Dense(units=16, activation='relu'),
    Dense(units=1)
])
model.compile(optimizer='adam', loss='mean_squared_error')

# Step 6: Train Model
model.fit(X, y, epochs=10, batch_size=32, verbose=1)

# Step 7: Backtesting with Dynamic TP/SL
initial_balance = 5000
balance = initial_balance
positions = []
returns = []

close_prices = data['Close'].values
volatilities = data['Volatility'].values
predicted_prices = []

for i in range(sequence_length, len(close_prices) - 1):
    current_sequence = scaled_data[i - sequence_length:i].reshape(1, sequence_length, 1)
    predicted_scaled = model.predict(current_sequence, verbose=0)
    predicted_price = scaler.inverse_transform(predicted_scaled)[0][0]
    predicted_prices.append(predicted_price)

    entry_price = close_prices[i]
    next_day_price = close_prices[i + 1]
    shares = balance / entry_price
    volatility = volatilities[i]

    take_profit_price = entry_price * (1 + 1.5 * volatility / entry_price)
    stop_loss_price = entry_price * (1 - 1.0 * volatility / entry_price)

    if predicted_price > entry_price:
        if next_day_price >= take_profit_price:
            exit_price = take_profit_price
        elif next_day_price <= stop_loss_price:
            exit_price = stop_loss_price
        else:
            exit_price = next_day_price

        pnl = (exit_price - entry_price) * shares
        balance += pnl
        returns.append(float(balance))
        positions.append((entry_price, exit_price, pnl))

# Step 8: Performance Metrics
returns_series = pd.Series([float(r) for r in returns])
daily_returns = returns_series.pct_change().dropna()

if not daily_returns.empty:
    sharpe_ratio = daily_returns.mean() / daily_returns.std() * np.sqrt(252)
else:
    sharpe_ratio = 0.0

cumulative = returns_series
rolling_max = cumulative.cummax()
drawdown = (cumulative - rolling_max) / rolling_max
max_drawdown = drawdown.min()

# Step 9: Display Results
backtest_df = pd.DataFrame(positions, columns=["Entry", "Exit", "PnL"])
backtest_df_tail = backtest_df.tail()

# Save CSV (Optional)
backtest_df.to_csv("NeuroSPYder_Backtest_Results.csv", index=False)

# Plot Equity Curve
plt.figure(figsize=(12, 5))
plt.plot(returns_series.values, label="Portfolio Value")
plt.title("📈 Equity Curve - NeuroSPYder")
plt.xlabel("Trades")
plt.ylabel("Portfolio Value")
plt.legend()
plt.grid(True)
plt.show()

# Plot Drawdown
plt.figure(figsize=(12, 4))
plt.plot(drawdown.values, label="Drawdown", color='red')
plt.title("📉 Drawdown Curve - NeuroSPYder")
plt.xlabel("Trades")
plt.ylabel("Drawdown %")
plt.legend()
plt.grid(True)
plt.show()

# Show final metrics and last trades
(sharpe_ratio, max_drawdown, balance), backtest_df_tail
